# Chess webservice examples

This notebook shows examples of using the CHESS-met data api to acess the [CHESS-met dataset](https://catalogue.ceh.ac.uk/documents/2ab15bf0-ad08-415c-ba64-831168be7293).  Please see [CHESS-met dataset](https://catalogue.ceh.ac.uk/documents/2ab15bf0-ad08-415c-ba64-831168be7293) for further information about this dataset.

Full documentation for the api is [here](https://data-eidc.ceh.ac.uk/1.0/2ab15bf0-ad08-415c-ba64-831168be7293/ui/).

**Note:** this workbook was developed using the [SciPy-notebook](https://hub.docker.com/r/jupyter/scipy-notebook) docker image from DockerHub.  If you have Docker run `docker run -p 8888:8888 jupyter/scipy-notebook:latest` to get a Jupyter notebook with all needed python dependencies already installed.

The following examples demonstrate all the api endpoints available, which are:
- reduceTime: summarises across time to produce a raster layer for a given region and date range
- reduceSpace: summarises across a region to produce single daily values for that region
- timeSeries: provides daily values for a point and date range
- subset: download a subset of the Chess data as a netcdf file

All examples use the surface temperature (abbreviated to `tas`) environmental variable, but there are 7 other variables to choose from (see [documentation](https://data-eidc.ceh.ac.uk/1.0/2ab15bf0-ad08-415c-ba64-831168be7293/ui/)).


## Set parameters and provide token
Start by setting the parameters needed by the `request`.  One of these parameters is an authentication token available from [https://data-eidc.ceh.ac.uk/authentication](https://data-eidc.ceh.ac.uk/authentication).  Once you have this token, run the cell below and paste it into the box as prompted:

In [ ]:
import requests

domain = 'https://data-eidc.ceh.ac.uk'
version = '1.0'
chess_dataset_identifier = '2ab15bf0-ad08-415c-ba64-831168be7293'

api_url_base = f'{domain}/{version}/{chess_dataset_identifier}/'

api_token = input('Paste your token (eg ff0fcc12db8efa2d71ab79e36335ca6c):  ')
# api_token = 'ff0fcc12db8efa2d71ab79e36335ca6c'

headers = {'Authorization': f'Bearer {api_token}'}

chess_variable = 'tas' #All examples use 'tas', which is the 'temperature at surface' in degrees Kelvin

print('Parameters have been set')

## reduceTime endpoint

The reduceTime endpoint subsets a Chess variable by space and time and averages (or min, max or count) across the time axis to produce a 2-d grid.  It returns data as json using the covjson format (see https://www.w3.org/TR/covjson-overview/ and https://covjson.org/)

In the example below, a bounding box around the Lake District is provided together with a date range, variable name (tas) and metric we want calcuated (mean).  So in this example, for each 1km square within the bounding box, we are asking for the mean tas value across all days within our date range.

The data are extracted from the response, put into a 2-d grid and plotted.  Also it is output to an asciigrid that can be viewed in a GIS.

In [ ]:
import json
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

# I've defined a bounding box of about 40x40km centred on Scafell Pike: 321495,507361
payload = {
    'dateFrom': '1961-01-01',
    'dateTo': '1962-01-01',
    'north': 547361,
    'south': 467361,
    'east': 361495,
    'west': 281495,
    'metric': 'mean'
}

# A function that requests and returns the data from the Chess api
def get_means_json():
    headers['Accept'] = 'application/json'
    api_url = f'{api_url_base}reduceTime/{chess_variable}'
    resp = requests.get(api_url, headers=headers, params=payload)
    if resp.status_code != 200:
        raise RuntimeError(f'GET /reduceTime/ {resp.status_code}')
    return resp

#get the data from the api and pull out the axis and values
resp = get_means_json()
# uncomment to see the json data returned, this is in covJson format: https://www.w3.org/TR/covjson-overview/ and https://covjson.org/
# print(json.dumps(resp.json(), indent=2))
resp_json = resp.json()
x_axis = resp_json['domain']['axes']['x']['values']
y_axis = resp_json['domain']['axes']['y']['values']
values = resp_json['ranges']['variable']['values']

# change the no data value (-99999) to python's not-a-number value (nan)
nodatavalue = -99999
values = np.array(values)
values[values==nodatavalue]=np.nan

# since the data are in covjson format, they are in one big long list
# reshape this into a 2-d numpy array of values, rotate and flip to get into correct orientation
data = np.array(values).reshape(len(x_axis),len(y_axis)).transpose(1,0)
data = np.flip(data)
data = np.flip(data, 1)

# display as a map
plt.imshow(data, interpolation='nearest')

# save as ascii grid that can be opened in qgis (would be nice to have this directly from api as a format option)
filename = 'asciigrid.csv'
data[np.isnan(data)]=nodatavalue
np.savetxt(filename, data, delimiter=' ', fmt='%6.5f')
cellsize = x_axis[1] - x_axis[0]
xllcorner = x_axis[0] - cellsize
yllcorner = y_axis[0] - cellsize
header = f'ncols {len(x_axis)}\nnrows {len(y_axis)}\nxllcorner {xllcorner}\nyllcorner {yllcorner}\ncellsize {cellsize}\nnodata_value {nodatavalue}\n'
with open(filename, 'r+') as f:
    content = f.read()
    f.seek(0,0)
    f.write(header + content)

print('finished - open asciigrid.csv in your GIS to view as a map - it is on the British National Grid')

## reduceSpace webservice

This averages across 1km grid squares for a region (user defined bounding box) for each day in a date range.
Essentially it is a timeseries for a bounding box of any size

In [ ]:
import json

# I've defined a bounding box of about 40x40km centred on Scafell Pike: 321495,507361
north = 547361
south = 467361
east = 361495
west = 281495
dateFrom = '1961-01-01'
dateTo = '1962-01-01'
metric = 'mean'
payload = {
    'dateFrom': dateFrom,
    'dateTo': dateTo,
    'north': north,
    'south': south,
    'east': east,
    'west': west,
    'metric': metric
}

# A function that requests and returns the data from the Chess api
def get_means_json():
    headers['Accept'] = 'application/json'
    api_url = f'{api_url_base}reduceSpace/{chess_variable}'
    resp = requests.get(api_url, headers=headers, params=payload)
    if resp.status_code != 200:
        raise RuntimeError(f'GET /reduceTime/ {resp.status_code}')
    return resp

#get the data from the api and pull out the axis and values
resp = get_means_json()
# uncomment to see the json data returned, this is in covJson format: https://www.w3.org/TR/covjson-overview/ and https://covjson.org/
# print(json.dumps(resp.json(), indent=2))
resp_json = resp.json()
values = resp_json['properties']['values']
print(f'These are the {metric} daily {chess_variable} values for your bounding box ({north}, {south}, {east}, {west}) across the date range {dateFrom} to {dateTo}:')
for val in values:
    print(f'{val["date"]} {val["mean"]}')


## timeSeries webservice
The time series webservice gets daily values for a point location for a date range

In [ ]:
import json

def get_timeseries(easting, northing, dateFrom, dateTo):
    payload = {
        'dateFrom': dateFrom,
        'dateTo': dateTo,
        'easting': easting,
        'northing': northing
    }
    print('Getting timeseries data...\n')
    headers['Accept'] = 'application/json'
    api_url = f'{api_url_base}timeseries/{chess_variable}'
    resp = requests.get(api_url, headers=headers, params=payload)
    if resp.status_code != 200:
        print(resp.text)
        raise RuntimeError(f'GET /timeSeries/ {resp.status_code}')
    return resp.json()
    
dateFrom = '1961-01-01'
dateTo = '1965-12-31'
easting = 290567
northing = 300123
timeseries_json = get_timeseries(easting, northing, dateFrom, dateTo)
print(f'These are the daily {chess_variable} values for your point ({easting}, {northing}) across the date range {dateFrom} to {dateTo}:')

# Turn the json into a flat table and force the date to be a 'real' date type ready for any timeseries analysis or plotting
import pandas as pd
from pandas import json_normalize
def to_table():
    timeseries = json_normalize(timeseries_json)
    timeseries['date'] = pd.to_datetime(timeseries['date'])
    return timeseries
timeseries_table = to_table()
print(timeseries_table)

# Plot it and calculate some statistics
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
plt.xticks(rotation=90)
plt.plot(timeseries_table['date'], timeseries_table['value'])
plt.title(f'Mean daily temperatures for the coordinate: {easting}, {northing} from {dateFrom} to {dateTo}')
print('\nHere are some statistics calculated across the data, followed by a chart of the data')
print(f'Mean: {timeseries_table["value"].mean()}')
print(f'Std: {timeseries_table["value"].std()}')
print(f'Min: {timeseries_table["value"].min()}')
print(f'Max: {timeseries_table["value"].max()}\n')

## subset webservice
The `subset` webservice provides a netcdf download of one Chess variable for a spatial-temporal filter.

The following two code blocks demonstrate firstly how to get a subset of Chess data and save as a netcdf file.  Then how to plot daily maps for the chess variable.

In [ ]:
# Get the netcdf subset for your region and date range and save as a temporary netcdf file.

import tempfile
import os

payload = {
    'dateFrom': '1961-01-01',
    'dateTo': '1961-02-28',
    'north': 250000,
    'south': 100000,
    'east': 650000,
    'west': 500000
}
   
def get_netcdf():
    print('Getting data...')
    headers['Content-Type'] = 'application/x-netcdf'
    api_url = f'{api_url_base}subset/{chess_variable}'
    resp = requests.get(api_url, headers=headers, params=payload, stream=True)
    if resp.status_code != 200:
        raise RuntimeError(f'GET /reduceTime/ {resp.status_code}')
    targetfile = tempfile.NamedTemporaryFile(delete=False, suffix='.nc')
    print(targetfile.name)
    with targetfile as output:
        output.write(resp.content)
    print(f'File location: {targetfile.name}')
    print(f'File size: {os.path.getsize(targetfile.name)}')
    return targetfile.name

subset_filename = get_netcdf()
print('done')

In [ ]:
# Now do something with the data you retrieved - in this case show daily maps for each date in the dataset.
# This has some hefty imports and that may need adding to your environment

%pip install netcdf4
import netCDF4
%pip install xarray
import xarray as xr
import matplotlib.pyplot as plt
    
def netcdf_maps():
    xr_dataset = xr.open_dataset(subset_filename)
    vmax = xr_dataset.max()['tas'].values
    vmin = xr_dataset.min()['tas'].values
    nrows = divmod(len(xr_dataset.time), 3)[0] + 1
    fig_height = nrows * 4
    fig, axes = plt.subplots(nrows=nrows, ncols=3, figsize=(16, fig_height))
    for i, date in enumerate(xr_dataset.coords['time']):
        x, y = divmod(i, 3)
        xr_dataset['tas'].sel(time=date).plot.pcolormesh(
            ax=axes[x, y], vmin=vmin, vmax=vmax, cmap='Spectral_r',
            add_colorbar=True, extend='both', add_labels=False)
        axes[x, y].set_title(str(date.values)[:10])
    for ax in axes.flat:
        ax.axes.get_xaxis().set_ticklabels([])
        ax.axes.get_yaxis().set_ticklabels([])
        ax.axes.axis('tight')
        ax.set_xlabel('')
    plt.tight_layout()
    fig.suptitle('Surface Air Temperature (K)', fontsize=16, y=1.02)
netcdf_maps()